In [2]:
import matplotlib.pyplot as plt
import numpy as np

from tracking.filters.jipdaf import PDA, Track, build_clusters, track_betas
from tracking.filters.dynamics_models import DefaultDynamicsPV2D, DefaultDynamicsPVA2D
from tracking.util.interpolator import LinearInterpolator, NodeCollection, SplineInterpolator
from tracking.util.metrics import RadarGenerator, Scan, TrackLog, PoissonClutter
from tracking.util.path import Path2D, PathFactory
from tracking.visualizer import TrackVisualizer
from tracking.filters.track_manager import TrackManager, TrackingVisualizer

%load_ext autoreload
%autoreload 2

In [4]:
TMAX = 500

nodes = [
    NodeCollection(np.array([[-400, 100],  [400, -100]]), [0, TMAX]),
    #NodeCollection(np.array([[-450, 0], [450, 0]]), [0, TMAX]),
    #NodeCollection(np.array([[3, 450], [-3, -450]]), [0, TMAX]),
    #NodeCollection(np.array([[-3, 450], [3, -450]]), [0, TMAX]),
]

node = NodeCollection(np.array(
    [[-400, -100], [-200, 25], [0, 0], [200, -25], [400, 100]]), np.linspace(0, TMAX, 5))
node2 = NodeCollection(np.array(
    [[400, 100], [200, -25], [0, 0], [-200, 25], [-400, -100]]), np.linspace(0, TMAX, 5))
node3 = NodeCollection(np.array(
    [[200, 300], [200, -25], [10, 5], [-200, 35], [-400, -100]]), np.linspace(0, TMAX, 5))
factory = PathFactory()
paths = factory.create(nodes, LinearInterpolator)
paths = factory.create([node, node2, node3], SplineInterpolator)

paths += factory.create([
    NodeCollection(np.array([
        [-400, 400], [-200, 300], [0, 400], [200, 300], [400, 200]
    ]), np.linspace(0, TMAX, 5)),

    NodeCollection(np.array([
        [-200, 300], [-100, 100], [0, 400], [200, 300], [400, 200]
    ]), np.linspace(0, TMAX, 5)),

    NodeCollection(np.array([
        [-200, 300], [-100, 0], [0, 400], [200, 300], [400, 200]
    ]), np.linspace(0, TMAX, 5)),
], SplineInterpolator)


nprobes = 200
probexy = np.zeros(nprobes)
probetimes = np.linspace(0.0, TMAX-0.1, nprobes)
radar = RadarGenerator(paths, 0.3, 1)
clutter_model = PoissonClutter([-400, 400, -220, 450], 0.0001)
scans = radar.make_scans_series(
    np.array([probetimes, probexy, probexy]).T, clutter_model)

manager = TrackManager(DefaultDynamicsPV2D())

nsteps = 60000
time = 0
dt = 0.05

working_scans = scans.copy()

for _ in range(nsteps):
    time += dt

    manager.predict_tracks(time, dt)

    if time+dt >= working_scans[0].time:
        time = working_scans[0].time

        manager.update_tracks(working_scans[0], time)
        manager.one_point_init(vmax=5, p0=0.35)
        manager.delete_false_tracks(.001)

        working_scans.pop(0)
        if len(working_scans) == 0:
            print("Processed all scans")
            break


number of active tracks: 0
number of active tracks: 54
number of active tracks: 92
number of active tracks: 77
number of active tracks: 65
number of active tracks: 68
number of active tracks: 76
number of active tracks: 80
number of active tracks: 70
number of active tracks: 73
number of active tracks: 80
number of active tracks: 73
number of active tracks: 65
number of active tracks: 65
number of active tracks: 72
number of active tracks: 75
number of active tracks: 66
number of active tracks: 70
number of active tracks: 73
number of active tracks: 70
number of active tracks: 69
number of active tracks: 74
number of active tracks: 75
number of active tracks: 62
number of active tracks: 65
number of active tracks: 72
number of active tracks: 73
number of active tracks: 69
number of active tracks: 67
number of active tracks: 69
number of active tracks: 70
number of active tracks: 76
number of active tracks: 74
number of active tracks: 63
number of active tracks: 54
number of active trac

In [ ]:
%autoreload 2
%matplotlib qt 


renderer = TrackingVisualizer(manager.logger)
plt.ion()
plt.figure()
plot = plt.subplot(1, 2, 1)
    
plot2 = plt.subplot(1, 2, 2)
renderer.filter_tracks_by_length(9)
renderer.render_epochs_in_range(plot2, -1, -1)
# renderer.set_number_rendered_epochs(0)
# renderer.render(plot2)

renderer.set_number_rendered_epochs(25)
renderer.clear_filter()
renderer.filter_tracks_by_length(4)

while renderer.advance_epoch():
    plot.cla()

    plot.set_ylim(-150, 450)
    plot.set_xlim(-450, 450)

    renderer.render_measurements(plot)
    renderer.render(plot)
    plt.show(block=False)
    plt.pause(0.1)


In [7]:
track_lengths = manager.logger.track_lengths()
track_lengths_filtered = manager.logger.display_track_lengths(10)

for t in track_lengths_filtered:
    exst_prob_hist = manager.logger.track_existence_probability_history(t)
    plt.plot(exst_prob_hist)

plt.show()

{59: {'length': 198}, 72: {'length': 198}, 85: {'length': 198}, 168: {'length': 194}, 264: {'length': 12}, 568: {'length': 175}, 629: {'length': 172}}


In [13]:
track_uid = 72
plt.figure()
ms_plot = plt.subplot(3, 1, 1)
exst_plot = plt.subplot(3, 1, 2)
track_plot = plt.subplot(3, 1, 3)
measurements_data = manager.logger.track_selected_measurements_history(track_uid)
ms_plot.plot([ep['num_actual_measurements'] for ep in measurements_data.values()])
ms.plot([ep['num_clutter_measurements'] for ep in measurements_data.values()])
plt.legend(['actual', 'clutter'])
plt.show()

exst_plot.plot(manager.logger.track_existence_probability_history(track_uid))

In [6]:
plt.hist(track_lengths, bins=200);
plt.show()